# Testing the Custom LLM Model

This notebook demonstrates how to use the trained Custom LLM model for text generation. We'll explore:

1. Loading a pretrained model checkpoint
2. Basic text generation with different parameters
3. Batch generation for multiple prompts
4. Effect of temperature and sampling parameters

In [1]:
import sys
sys.path.append('..') 

import torch
from transformers import AutoTokenizer
from src.inference.inference_pipeline import InferencePipeline

# Set random seed for reproducibility
torch.manual_seed(42)

c:\Git\Custom-LLM\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Model and Tokenizer

First, we'll load a pretrained tokenizer and our trained model checkpoint.

In [2]:
# Initialize tokenizer (update with your preferred base model)
tokenizer = AutoTokenizer.from_pretrained('gpt2')  # Example using GPT-2 tokenizer

# Load the model checkpoint
model_path = '../checkpoints/model.pt'  # Update with your model path
pipeline = InferencePipeline.from_pretrained(
    model_path=model_path,
    tokenizer=tokenizer,
    model_config={    
                'd_model': 512,  # Smaller dimension
                'n_heads': 8,   # Fewer attention heads
                'n_layers': 4,  # Fewer layers
                'd_ff': 2048,   # Smaller feed-forward dimension
                'dropout': 0.1
    }
)

## Basic Text Generation

Let's try generating text with default parameters.

In [47]:
prompt = "Albert Einstein was born in"

# Generate with default parameters
generated_text = pipeline.generate(
    prompt=prompt,
    max_length=100,
    num_return_sequences=1
)

print(f"Prompt: {prompt}")
print(f"Generated: {generated_text[0]}")

Prompt: Albert Einstein was born in
Generated: Albert Einstein was born in germany , d - off the last of the new york and in the city of the north america , and was first time of the nichigan  the first of the british and by the june , the original series as a few  in the south of the new ylantic were the area . it was in the game . the united states  , the two british of this time of the main 's. and three years .


## Exploring Temperature Effects

Now let's see how different temperature values affect the output. Higher temperature (>1.0) makes the output more random, while lower temperature (<1.0) makes it more focused and deterministic.

In [4]:
prompt = "In the year 2050, robots will"

# Try different temperatures
temperatures = [0.5, 1.0, 1.5]

for temp in temperatures:
    print(f"Temperature: {temp}")
    generated = pipeline.generate(
        prompt=prompt,
        max_length=100,
        temperature=temp,
        num_return_sequences=1
    )
    print(f"Generated: {generated[0]}")

Temperature: 0.5
Generated: In the year 2050, robots williams of the first - yard line of the " , the film 's.s.s.s "
Temperature: 1.0
Generated: In the year 2050, robots will and is the united states is a nba of his first time , his son of a large for a book , which israel  .
Temperature: 1.5
Generated: In the year 2050, robots will she is not become a " a three their final but had little to be able to its own , with other at least the film 'm that one " and some of the area a member that the time or " that a total is based by all he 'mal season .


## Top-k and Top-p Sampling

Let's experiment with different sampling strategies using top-k and nucleus (top-p) sampling.

In [5]:
prompt = "The future of space exploration"

# Different sampling configurations
configs = [
    {'top_k': 50, 'top_p': 0.9},
    {'top_k': 10, 'top_p': 0.9},
    {'top_k': 50, 'top_p': 0.5}
]

for config in configs:
    print(f"Config: {config}")
    generated = pipeline.generate(
        prompt=prompt,
        max_length=100,
        temperature=1.0,
        **config
    )
    print(f"Generated: {generated[0]}")

Config: {'top_k': 50, 'top_p': 0.9}
Generated: The future of space exploration for the " of the nlotte , and the west and then the last german  " , the other and the main in the song in the city , where the hms , it "  h  1 .
Config: {'top_k': 10, 'top_p': 0.9}
Generated: The future of space exploration .
Config: {'top_k': 50, 'top_p': 0.5}
Generated: The future of space exploration


## Batch Generation

Finally, let's try generating multiple sequences in batch mode.

In [6]:
prompts = [
    "The key to successful AI development is",
    "In the next decade, quantum computing will",
    "The relationship between humans and AI will"
]

# Generate multiple sequences in batch
batch_generated = pipeline.batch_generate(
    prompts=prompts,
    batch_size=2,
    max_length=100,
    temperature=0.8,
    top_k=40,
    top_p=0.9
)

for prompt, generated in zip(prompts, batch_generated):
    print(f"Prompt: {prompt}")
    print(f"Generated: {generated}")

Prompt: The key to successful AI development is
Generated: The key to successful AI development is the song .
Prompt: In the next decade, quantum computing will
Generated: In the next decade, quantum computing williams.  1  , which is a  . he is a result in the united states .
Prompt: The relationship between humans and AI will
Generated: The relationship between humans and AI william
